In [159]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as ex
import plotly.graph_objects as go
import scipy.linalg as la
from scipy.fftpack import dct, idct
import _pickle as pickle
import cv2 as cv2
import os
from pypapi import events, papi_high as high
import time
import scipy.stats

from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection as GRP
from sklearn.random_projection import SparseRandomProjection as SRP

In [130]:
def get_pca(M, axis=0):
    cov = M.T @ M
    if axis==1:
        cov = M @ M.T

    cov = cov / (M.shape[1]-1)
    s, u = la.eigh(cov)
    u /= np.linalg.norm(u, axis=axis)

    return lambda k: M @ u[:, -k:] if axis==0 else M @ u[-k:, :].T

def get_dct(M, axis=0):
    D = dct(M, axis=axis)
    sum_features = np.sum(D,axis=axis)
    idx = np.argsort(sum_features)
    return lambda k: idct(D[:, idx[:k]],axis=axis) if axis==0 else idct(D[idx[:k], :],axis=axis)

def rp(M, axis=0):
    R = np.random.normal(0, 1, (M.shape[1],M.shape[1]))
    if axis==1:
        R = np.random.normal(0, 1, (M.shape[0],M.shape[0]))
    
    R /= np.linalg.norm(R, axis=axis)
    return lambda k: M @ R[:, :k] if axis==0 else R[:k, :] @ M

def srp(M, axis=0):
    R = np.random.uniform(0,1, (M.shape[1], M.shape[1]))
    if axis==1:
        R = np.random.uniform(0,1, (M.shape[0], M.shape[0]))
    R2 = np.copy(R)
    R2[R < 1/6.0] = 1
    R2[R > 5/6.0] = -1
    R2[np.abs(R2) != 1] = 0
    R2*= np.sqrt(3)
    return lambda k: M @ R2[:, :k] if axis==0 else R2[:k, :] @ M

def rp2(M, k):
    return GRP(n_components=k).fit_transform(M)
def srp2(M, k):
    return SRP(n_components=k).fit_transform(M)


In [164]:
def rand_measure(x, frac=1, num=100, fn="l2-norm"):
    arr = np.random.permutation(x)
    if arr.shape[0] > num:
        arr = arr[:num, :]
        n = int(num / 2)
    else:
        n = int(arr.shape[0] / 2)
    if fn == "l2-norm":
        return np.sqrt(frac) * np.mean(np.linalg.norm(arr[:n, :] - arr[n:,:], axis=0))
    else:
        return np.mean(np.sum(arr*arr,axis=1))

def measure(x, frac=1, fn="l2-norm"):
    d = np.diff(x, axis=0)
    if fn == "l2-norm":
        return mean_confidence_interval(np.sqrt(frac) * np.linalg.norm(d, axis=1))
    else:
        return mean_confidence_interval((np.sum(x*x, axis=1)))
def norm(x, axis=1):
    if axis!=1:
        return (x-np.mean(x)) / np.std(x)
    else:
        return (x-np.mean(x, axis=axis, keepdims=True)) / np.std(x, axis=axis, keepdims=True)
def timer(f):
    if f == 0:
        return time.time()
    else:
        return time.time() - f
        


def mean_confidence_interval(data, confidence=0.95):
    a = data
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


# Test on Image data 

In [23]:
def unpickle(file):
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='bytes')
    return d

def load(num=1):
    data_p = []
    for dname, dirs, files in os.walk("../data/cifar-10-python"):
        for fname in files:
            if "data_batch" in fname:
                fpath = os.path.join(dname, fname)
                data_p.append(fpath)
    print(data_p)
    return [unpickle(p) for p in data_p]

d = load()

['../data/cifar-10-python/cifar-10-batches-py/data_batch_4', '../data/cifar-10-python/cifar-10-batches-py/data_batch_2', '../data/cifar-10-python/cifar-10-batches-py/data_batch_1', '../data/cifar-10-python/cifar-10-batches-py/data_batch_5', '../data/cifar-10-python/cifar-10-batches-py/data_batch_3']


In [5]:
for keys, _ in d[0].items():
    print(keys)

b'batch_label'
b'labels'
b'data'
b'filenames'


In [204]:
N = 100
np.random.seed(2020)

data = d[0][b"data"][:N].astype(np.float32).reshape((N, 32,32,3))
data = np.asarray([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in data])
data = np.reshape(data, (N, -1))
data = norm(data)

n = data.shape[1]
ks = np.arange(1, 50)

o_diff, _, _ = measure(data)
p_diff, d_diff, r_diff, sr_diff = [],[],[],[]
p_min, d_min, r_min, sr_min = [],[],[],[]
p_max, d_max, r_max, sr_max = [],[],[],[]
p_time, d_time, r_time, sr_time = [],[],[],[]

t = 0
for k in ks:
    frac = n / k

    t = timer(0)
    p = get_pca(data)(k)
    t = timer(t)
    m, mmin, mmax = measure(norm(p), frac=frac)
    p_diff.append(m)
    p_min.append(mmin)
    p_max.append(mmax)
    p_time.append(t)

    t = timer(0)
    p = get_dct(data)(k)
    t = timer(t)    
    m, mmin, mmax = measure(norm(p), frac=frac)
    d_diff.append(m)
    d_min.append(mmin)
    d_max.append(mmax)
    d_time.append(t)

    t = timer(0)
    p = rp2(data,k)
    t = timer(t)    
    m, mmin, mmax = measure(norm(p), frac=frac)
    r_diff.append(m)
    r_min.append(mmin)
    r_max.append(mmax)
    r_time.append(t)

    t = timer(0)
    p = srp2(data,k)
    t = timer(t)    
    m, mmin, mmax = measure(norm(p), frac=frac)
    sr_diff.append(m)
    sr_min.append(mmin)
    sr_max.append(mmax)
    sr_time.append(t)

p_diff = np.asarray(p_diff) - o_diff
d_diff = np.asarray(d_diff) - o_diff
r_diff = np.asarray(r_diff) - o_diff
sr_diff = np.asarray(sr_diff) - o_diff

p_min  = np.asarray(p_min) - o_diff
d_min  = np.asarray(d_min) - o_diff
r_min  = np.asarray(r_min) - o_diff
sr_min = np.asarray(sr_min) - o_diff

p_max   = np.asarray(p_max) - o_diff
d_max   = np.asarray(d_max) - o_diff
r_max   = np.asarray(r_max) - o_diff
sr_max = np.asarray(sr_max) - o_diff

    




In [206]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ks, y=p_diff, name="PCA", marker_symbol="circle", mode="markers", error_y=dict(type="data", symmetric=False, array=p_max, arrayminus=p_min)))
# fig.add_trace(go.Scatter(x=ks, y=p_diff, name="PCA", marker_symbol="circle", mode="markers"))#, error_y=dict(type="data", symmetric=False, array=p_max, arrayminus=p_min)))
# fig.add_trace(go.Scatter(x=list(ks)+list(ks[::-1]), y=list(p_max)+list(p_min[::-1]), hoverinfo="skip", showlegend=False))
# fig.add_trace(go.Scatter(x=list(ks)+list(ks[::-1]), y=list(p_max)+list(p_min[::-1]), fill="toself", fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", showlegend=False))

fig.add_trace(go.Scatter(x=ks, y=d_diff, name="DCT", marker_symbol="square", mode="markers", error_y=dict(type="data", symmetric=False, array=d_max, arrayminus=d_min)))
# fig.add_trace(go.Scatter(x=ks, y=d_diff, name="DCT", marker_symbol="square", mode="markers"))##, error_y=dict(type="data", symmetric=False, array=d_max, arrayminus=d_min)))
# fig.add_trace(go.Scatter(x=list(ks)+list(ks[::-1]), y=list(d_max)+list(d_min[::-1]), hoverinfo="skip", showlegend=False))
# fig.add_trace(go.Scatter(x=list(ks)+list(ks[::-1]), y=list(d_max)+list(d_min[::-1]), fill="toself", fillcolor='rgba(0,0,80,0.2)', line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", showlegend=False))

fig.add_trace(go.Scatter(x=ks, y=r_diff, name="RP", marker_symbol="cross", mode="markers", error_y=dict(type="data", symmetric=False, array=r_max, arrayminus=r_min)))
# fig.add_trace(go.Scatter(x=ks, y=r_diff, name="RP", marker_symbol="cross", mode="markers"))#, error_y=dict(type="data", symmetric=False, array=r_max, arrayminus=r_min)))
# fig.add_trace(go.Scatter(x=list(ks)+list(ks[::-1]), y=list(r_max)+list(r_min[::-1]), fill="toself", fillcolor='rgba(100,0,80,0.2)', line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", showlegend=False))

fig.add_trace(go.Scatter(x=ks, y=sr_diff, name="SRP", marker_symbol="diamond", mode="markers", error_y=dict(type="data", symmetric=False, array=sr_max, arrayminus=sr_min)))
# fig.add_trace(go.Scatter(x=ks, y=sr_diff, name="SRP", marker_symbol="diamond", mode="markers"))#, error_y=dict(type="data", symmetric=False, array=sr_max, arrayminus=sr_min)))
# fig.add_trace(go.Scatter(x=list(ks)+list(ks[::-1]), y=list(sr_max)+list(sr_min[::-1]), fill="toself", fillcolor='rgba(100,100,80,0.2)', line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", showlegend=False))

fig.update_layout(
    title_text="Euclidean error using RP, SRP, PCA and DCT",
    width=800,
)

fig.update_xaxes(title_text='Reduced dimension')
fig.update_yaxes(title_text='Error')

fig.show()

In [82]:
N = 2
data = d[0][b"data"][:N].astype(np.float32).reshape((N, 32,32,3))
data = np.asarray([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in data])
data = np.reshape(data, (N, -1))
data = norm(data)

k = 38
p = get_pca(data)(k)
dc = norm(get_dct(data)(k))
r = norm(rp(data)(k))
sr = norm(srp(data)(k))
# p_diff.append(measure(norm(p), frac=frac))
# p_time.append(t)

# t = timer(0)
# p = get_dct(data)(k)
# t = timer(t)
# d_diff.append(measure(norm(p), frac=frac))
# d_time.append(t)

# t = timer(0)
# p = rp(data)(k)
# t = timer(t)
# r_diff.append(measure(norm(p), frac=frac))
# r_time.append(t)

# t = timer(0)
# p = srp(data)(k)
# t = timer(t)
# sr_diff.append(measure(norm(p), frac=frac))
# sr_time.append(t)

In [83]:
calc = lambda x: np.sqrt(2) * np.sum((x[0]-x[1])**2)
o_diff = calc(data)
p_diff = calc(p)
d_diff = calc(dc)
r_diff = calc(r)
sr_diff = calc(sr)
print(o_diff)
print(p_diff)
print(d_diff)
print(r_diff)
print(sr_diff)


2335.967302619788
2335.967993153754
141.65941782208887
71.16310311338175
98.90207241245112
